<a href="https://colab.research.google.com/github/RaziehAkbari2020/Search_Tavily-with-Langchain/blob/main/Tavily_with_Scrapping_version_1_17_7_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tavily-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00


In [2]:
!pip install requests beautifulsoup4 pdfplumber


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 753.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 81.2 MB/s eta 0:00:00


In [3]:
from tavily import TavilyClient
import requests
from bs4 import BeautifulSoup
import pdfplumber
from io import BytesIO
import os
import re

# دریافت ورودی از کاربر
university_name = input("نام دانشگاه را وارد کنید: ")
program_name = input("نام برنامه را وارد کنید: ")

# کلید API برای Tavily
tavily = TavilyClient(api_key="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")

# جستجو با استفاده از Tavily
query = f"""
    I want to apply for Master of {program_name} in {university_name} for international or non-European applicants.
    What is the deadline exactly?
    What are the language requirements?
    What is exactly for language score?
    What is the GPA for Bachelor's degree?
    Please search only from the mentioned university
"""

response = tavily.search(
    query=query,
    search_depth="advanced",
    include_raw_content=True
)

# استخراج محتوا و لینک‌ها از نتایج جستجو
context = [{"url": obj["url"], "content": obj["content"]} for obj in response["results"]]

# ساخت پوشه برای ذخیره فایل‌ها
output_dir = "/content/scraped_content"
os.makedirs(output_dir, exist_ok=True)

pdf_links = []

def scrape_web_content(url, original_content):
    """Scrape content from the given URL"""
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return "Failed to retrieve the webpage."

    soup = BeautifulSoup(response.text, 'html.parser')
    page_text = extract_text_from_soup(soup)
    combined_content = f"Original Content:\n{original_content}\n\nExtracted Content:\n{page_text}"

    pdf_links.extend(find_pdf_links(soup, url))

    webpage_filename = f'webpage_content_{url_to_filename(url)}.txt'
    save_to_file(webpage_filename, combined_content)
    print(f"متن صفحه وب ذخیره شد: {webpage_filename} - لینک: {url}")
    display_file_link(webpage_filename)

    for idx, pdf_link in enumerate(pdf_links):
        pdf_filename = f'pdf_content_{url_to_filename(pdf_link)}_{idx + 1}.txt'
        pdf_content = scrape_pdf_content(pdf_link)
        save_to_file(pdf_filename, pdf_content)
        print(f"متن پی‌دی‌اف ذخیره شد: {pdf_filename} - لینک: {pdf_link}")
        display_file_link(pdf_filename)

def extract_text_from_soup(soup):
    """Extract text from BeautifulSoup object"""
    page_content = []
    for tag in soup.find_all(['h1', 'h2', 'h3', 'p']):
        text = tag.get_text(" ", strip=True)
        page_content.append(text)

    page_content = "\n".join(page_content)
    return page_content

def find_pdf_links(soup, base_url):
    """Find PDF links in the BeautifulSoup object"""
    pdf_links = []
    base_url = re.match(r'^(https?://[^/]+)', base_url).group(1)
    for link in soup.find_all('a', href=True):
        href = link['href']
        if href.endswith('.pdf'):
            pdf_link = href if href.startswith('http') else base_url + href
            pdf_links.append(pdf_link)
    return pdf_links

def scrape_pdf_content(pdf_url):
    """Scrape content from the given PDF URL"""
    response = requests.get(pdf_url)
    if response.status_code != 200:
        return "Failed to retrieve the PDF."

    text_content = []
    try:
        with pdfplumber.open(BytesIO(response.content)) as pdf:
            for page in pdf.pages:
                text_content.append(page.extract_text())
        return " ".join(text_content)
    except Exception as e:
        return f"Error reading PDF: {e}"

def save_to_file(filename, content):
    """Save content to a file"""
    filepath = os.path.join(output_dir, filename)
    with open(filepath, 'w', encoding='utf-8') as file:
        file.write(content)

def display_file_link(filename):
    """Display link to download file"""
    filepath = os.path.join(output_dir, filename)
    from google.colab import files
    files.download(filepath)

def url_to_filename(url):
    """Convert a URL to a filename-friendly string"""
    return re.sub(r'[^a-zA-Z0-9]', '_', url)

# اسکرپ کردن محتوا از لینک‌های استخراج‌شده
for item in context:
    scrape_web_content(item["url"], item["content"])

# نمایش لینک‌های PDF
for idx, pdf_link in enumerate(pdf_links):
    print(f"PDF {idx + 1}: {pdf_link}")


نام دانشگاه را وارد کنید: Technical University of Darmstadt
نام برنامه را وارد کنید: Autonomous systems and robots
متن صفحه وب ذخیره شد: webpage_content_https___www_tu_darmstadt_de_studieren_studieninteressierte_studienangebot_studiengaenge_studiengang_181440_en_jsp.txt - لینک: https://www.tu-darmstadt.de/studieren/studieninteressierte/studienangebot_studiengaenge/studiengang_181440.en.jsp


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

متن پی‌دی‌اف ذخیره شد: pdf_content_https___www_tu_darmstadt_de_media_daa_responsives_design_02_studium_medien_01_studieninteressierte_medien_02_studienangebot_medien_master_of_science_1_autonome_systeme_2_studienplan_131_msc_autonome_systeme_pdf_en_pdf_1.txt - لینک: https://www.tu-darmstadt.de/media/daa_responsives_design/02_studium_medien/01_studieninteressierte_medien/02_studienangebot_medien/master_of_science_1/autonome_systeme_2/studienplan_131/msc_autonome_systeme_pdf.en.pdf


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

متن پی‌دی‌اف ذخیره شد: pdf_content_https___www_tu_darmstadt_de_media_daa_responsives_design_02_studium_medien_01_studieninteressierte_medien_02_studienangebot_medien_master_of_science_1_autonome_systeme_2_studienplan_131_msc_autonome_systeme_pdf_en_pdf_2.txt - لینک: https://www.tu-darmstadt.de/media/daa_responsives_design/02_studium_medien/01_studieninteressierte_medien/02_studienangebot_medien/master_of_science_1/autonome_systeme_2/studienplan_131/msc_autonome_systeme_pdf.en.pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

PDF 1: https://www.tu-darmstadt.de/media/daa_responsives_design/02_studium_medien/01_studieninteressierte_medien/02_studienangebot_medien/master_of_science_1/autonome_systeme_2/studienplan_131/msc_autonome_systeme_pdf.en.pdf
PDF 2: https://www.tu-darmstadt.de/media/daa_responsives_design/02_studium_medien/01_studieninteressierte_medien/02_studienangebot_medien/master_of_science_1/autonome_systeme_2/studienplan_131/msc_autonome_systeme_pdf.en.pdf
